**version 5.3**

# Basic output handling

This example shows how to access the data that is stored in HDF5.

1. <a href='#iniandendstate'>Ini- and endstate</a>
2. <a href='#orbit'>Orbit</a>
3. <a href='#ascotpy'>ascotpy</a>

### Necessary imports

In [1]:
import numpy as np
from a5py.ascot5io.ascot5 import Ascot
from a5py.ascotpy import Ascotpy

<a id='iniandendstate'></a>


## Ini- and endstate

Begin by generating the sample Ascot5 data file with  
`python a5py/preprocessing/simpleruns.py`  
and carrying out the simulation as  
`./ascot5_main --in=helloworld.h5` .

Open Python session and read the file:

In [ ]:
h5 = Ascot("helloworld.h5")
endstate = h5.active.endstate

Endstate (inistate) containts the position the markers have at the end (beginning) of the simulation. The data is accessed via brackets and it is ordered with respect to marker ID:

In [ ]:
endstate["r"]

In [ ]:
endstate["id"]

### Guiding center and particle data

The endstate contains both guiding-center and particle (i.e. gyro-orbit) coordinates irrespective what simulation mode was used. To access particle data, add suffix "prt" to the queried quantity. If no suffix is given, guiding-center data is returned.

In [ ]:
endstate["rprt"]

It is a good idea to always query the quantities that correspond to the simulation mode, as there can be a significant deviation in e.g. energy if the simulation was done in GO mode and the guiding-center approximation is not valid. In this example it does not matter which is used:

In [ ]:
endstate["energy"][0]

In [ ]:
endstate["energyprt"][0]

The data can also be accessed via `get`function that allows to pick markers with specific ID and end condition (see `a5py/marker/endcond.py` for a list of different end conditions):

In [ ]:
endstate.get("r", ids=np.arange(1,20), endcond="wall")

### Derived quantities

Not all data is stored in the HDF5 file but some is evaluated run-time when the data is queried. For example, the HDF5 file contains only the cylindrical coordinates but when one queries for cartesian coordinates, the coordinate transformation is done behind the curtains (the evaluation of these derived quantities is done in `a5py/physlib/evaluate.py`):

In [ ]:
endstate["x"][0]

Furthermore, the endstate can be used to evaluate the background data at the guiding center position (or at the particle position when suffix "prt" is used). Also the background data is evaluated in the process if the queried data requires its evaluation e.g. to evaluate toroidal canonical angular momentum one needs to know value of the psi and the magnetic field.

**IMPORTANT**  
The background data that is used in these evaluations is always the one that is set as active in the HDF5 file and **not** the one that was used in the simulation. This speeds the evaluation process as one can make a magnetic field with sparse grid (or a 2D field) for postprocessing if the simulation requires dense 3D grid.

Whenever background data is used in evaluation, you will see the initialization process printed out in the terminal:

In [ ]:
endstate["psi"][0]

### Units

As you may have noticed by now, the output arrays are not typical numpy arrays but unyt arrays that contain both quantity and unit. The units are provided via [unyt](https://pypi.org/project/unyt/) package which provides a documentation.

Typically unyt arrays behave exactly like numpy arrays but this is not always the case. To strip unyt array of its quantity, thus converting it to a numpy array, simply use attribute `.v`:

In [ ]:
endstate["psi"][0].v

The easiest way to use unyt is to first convert the quantity to desired units before stripping them. Unit conversions can be done with `to` function or to a predefined base, e.g. SI system. There is also an ascot unit system.

In [ ]:
energy = endstate["energy"][0]
print(energy)
print(energy.to("J")) # Convert to a specific unit (returns a new value)
energy.convert_to_mks() # Convert to SI units (the variable is converted)
print(energy)
energy.convert_to_base("ascot") # Convert to ascot units (the variable is converted)
print(energy)

### Available quantities

The list of available quantities is:

| Quantity | Comment |
|:--|:--|
| r, phi, phimod, z | Cylindrical coordinates |
| v_r, v_phi, v_z, vnorm and p_r, p_phi, p_z, pnorm | Velocity and momentum components |
| mu, vpar and ppar, zeta | Magnetic moment, parallel velocity and momentum, gyroangle |
| rho, theta, thetamod | Square root of the normalized poloidal flux and the poloidal angle |
| psi, Br, Bphi, Bz, Bnorm | Poloidal flux and the magnetic field components and magnitude |
| time, mileage, cputime | Laboratory time, marker time and CPU time. See explanation below |
| weight | Marker weight (how many physical particles/what particle flux a marker represents) |
| mass, charge, anum, znum | Mass, charge, mass number, and charge number |
| ptor | Toroidal canonical angular momentum |
| endcond | The condition at which marker was terminated |
| energy, gamma | Kinetic energy and Lorentz factor. Energy, like other quantities, include relativistic corrections. |
| id | Unique identifier number for each marker |

**phimod vs phi**  
Phi is the cumulative toroidal angle (how much in total the marker has advanced toroidally) while phimod is `mod(phi, 360 deg)`. This same applies for the poloidal angle as well.

**time vs mileage vs cputime**  
Time is the laboratory time, or one can think of it as the pulse duration. Corresponds to the time grid in which time-dependent backgrounds are given.

Mileage is the time (coordinate) noting for how far in time the marker has been integrated (traced). Field line markers don't advance in time (their time remains constant) so for those the mileage measures how long in distance the field line has been integrated. However, for consistence the mileage in output is this distance divided by the speed of light to get units of time. Whereas markers can have different initial time coordinates, their mileage always begins at zero.

CPU time is the processor has spent simulating a marker. Note that the sum of the total CPU time calculated from the endstate does not equal to the time the whole simulation lasted (even if we don't count input initialization and output writing) as markers are simulated in parallel. The total simulation time can be estimated as `sum_of_cpu_time / number_of_markers_run_in_parallel`.

**Aliases**  
Accessing different quantities is case insensitive, and all spaces and underscores are ignored. Furthermore, people have different preferences and sometimes it is better to be more verbose, which is why quantities have different aliases (defined in `a5py/physlib/alias.py`). All of the following yield same result:

In [ ]:
print(endstate["theta"][0])
print(endstate["pol"][0])
print(endstate["poloidal angle"][0])
print(endstate["poloidal_angle"][0])

### Data access for developers

It is possible to just read the raw data as

In [ ]:
endstate.read()

which reads the contents of the HDF5 file to a dictionary without any modifications (note that the data can be unordered). Naturally the derived quantities cannot be accessed this way (since they are not stored) and this raw access is discouraged (because the contents of the file might change between different versions). For debugging purposes `read` function is suitable.

If your script is handling large datasets, you can access the data as you would access an ordinary HDF5 file which allows you to e.g. read the data in chunks or read just a particular dataset:

In [ ]:
with endstate as e:
    print(e["r"])

<a id='orbit'></a>


## Orbit data

The orbit data contains marker coordinates collected along the marker orbit, either between given mileage intervals or when a pre-defined toroidal or poloidal plane is crossed (Poincaré data).

In [ ]:
h5 = Ascot("helloworld.h5")
orbit = h5.active.orbit

The orbit data is accessed in a similar manner as the int- and endstate data is and it also provides access to derived and background quantities. However, there are some notable differences.

The data is sorted first by ID and then by time:

In [ ]:
print(orbit["ids"][:3000:100])
print(orbit["time"][:3000:100])

If Poincaré data was collected, the data contains additional field `pncrid` which identifies the Poincaré plane that was crossed when that data point was recorded. The `get` function allows parsing by `pncrid`. Note that parsing by end condition returns the whole orbit data corresponding to markers whose simulation ended in that condition.

Another additional field is `simmode` which tells the active simulation mode when that particular point was recorded. Note that in hybrid mode it is possible for the simulation mode to change between the simulation. This field is used to identify in what coordinate system the derived quantities are evaluated. Here it is not possible to use suffix "prt" to identify particle data as the evaluated quantities are always in that system that corresponds to the simulation mode. In other words, "energy" returns particle energy if simulation mode was GO, guiding center energy in GC simulation, either one in the hybrid mode (varies between the data points), and some mess in the field line simulation.

<a id='ascotpy'></a>


## Ascotpy

Postprocessing in ASCOT5 relies heavily on the Python interface (*ascotpy*) to ASCOT5 library (*libascot*). With ascotpy, it is possible to evaluate background data using exactly the same methods the code itself uses.

Previous examples have shown how to evaluate e.g. psi at the marker location using the endstate, but how to evaluate it at an arbitrary location? This requires using ascotpy explicitly and the instructions are given here.

Begin by initializing the Ascotpy object:

In [ ]:
a5 = Ascotpy("helloworld.h5")

If an error is raised at this stage, either the HDF5 file is missing, `libascot.so` has not been compiled or it is not found in `LD_LIBRARY_PATH`.

Next step is to initialize the input fields. In this example, we wish to evaluate EM-fields so we need to initialize `bfield` and `efield` inputs:

In [ ]:
h5  = Ascot("helloworld.h5")
qid = h5.active.bfield.get_qid()

a5.init(bfield=qid, efield=True)

Note that we have used QID to specify which input we want to initialize. If no QID is given, then the field that is currently set as active is initialized.

Input evaluation is done with the `evaluate` function. The data can be queried either by giving a set of coordinates where the data is to be evaluated, or by giving arrays that define a grid and the data is evaluated at the grid points in which case keyword argument `grid=True` is used.

In [ ]:
print(a5.evaluate(
            R=np.array([6, 7]), phi=np.array([0, 0]), z=np.array([1, 1]), t=np.array([0, 0]), 
            quantity="bnorm") )
print("")
print(a5.evaluate(
            R=np.array([6, 7]), phi=np.array([0, 0]), z=np.array([1, 1]), t=np.array([0, 0]), 
            quantity="bnorm", grid=True) )

Once we are done with the evaluation, we can deallocate memory by freeing the inputs (here QID is not used):

In [ ]:
a5.free(bfield=True, efield=True)

**Important**  
Do not have more than one Ascotpy object initialized at any moment! Memory allocation for the inputs is done in the C side, so e.g. initializing magnetic field in two Ascotpy objects causes the field in the first to become overwritten.

### Note to developers

ascotpy is contained in `a5py/ascotpy`. The main file which defines Ascotpy object is `ascotpy.py`, while each input is assigned their own `lib*.py` file and Ascotpy inherits them all. The actual Python interface to `libascot.so` is contained in `libascot.py`.